Now after cleaning the data we will move to feature enginnering to get meaning full pca analysis and the clustering 

In [219]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler


# Load the cleaned unscaled data
df = pd.read_csv("cleaned_unscaled.csv")
print("Loaded shape:", df.shape)
df

Loaded shape: (52585, 50)


,Energy Consumption (kWh),Temperature (°C),Humidity (%),Occupancy Rate (%),Lighting Consumption (kWh),HVAC Consumption (kWh),Energy Price ($/kWh),Carbon Emission Rate (g CO2/kWh),Power Factor,Voltage Levels (V),...,Building Orientation_West,Carbon Emission Reduction Category_Low Reduction,Carbon Emission Reduction Category_Moderate Reduction,Carbon Emission Reduction Category_No Reduction,hour_sin,hour_cos,month_sin,month_cos,day_sin,day_cos
0,74.679912,31.357437,62.472886,49.293544,9.892054,9.073339,0.053297,341.764320,0.932078,237.501909,...,False,False,True,False,0.000000,1.000000,0.0,1.000000,2.012985e-01,0.97953
1,46.592761,30.229121,63.067770,65.039117,11.063696,26.487881,0.019031,427.270057,0.984152,221.200139,...,False,False,True,False,0.258819,0.965926,0.0,1.000000,2.012985e-01,0.97953
2,58.836657,19.182581,65.030972,0.000000,0.582297,10.385565,0.060282,278.064969,0.800676,237.603425,...,False,False,True,False,0.500000,0.866025,0.0,1.000000,2.012985e-01,0.97953
3,53.586516,16.700048,67.405643,27.398152,3.580005,8.199913,0.209318,691.906306,0.921343,224.560001,...,False,False,False,True,0.707107,0.707107,0.0,1.000000,2.012985e-01,0.97953
4,37.800448,29.620051,55.069770,74.220380,17.823608,12.268133,0.225335,620.526286,0.901588,229.243806,...,False,False,True,False,0.866025,0.500000,0.0,1.000000,2.012985e-01,0.97953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52580,78.697373,18.669766,76.145160,86.565125,11.475367,13.489796,0.119900,499.016888,0.869080,230.541960,...,False,True,False,False,-0.866025,0.500000,-0.5,0.866025,-2.449294e-16,1.00000
52581,59.400463,24.814296,93.473230,51.380712,18.418070,25.319140,0.180057,344.329532,1.000000,234.583182,...,False,True,False,False,-0.707107,0.707107,-0.5,0.866025,-2.449294e-16,1.00000
52582,65.395535,3.952222,75.281979,49.625698,17.637973,19.546613,0.168849,317.509517,0.768559,220.135476,...,False,False,True,False,-0.500000,0.866025,-0.5,0.866025,-2.449294e-16,1.00000
52583,58.505085,12.342418,54.181047,87.792609,4.205146,0.000000,0.139890,183.725258,0.858071,232.386000,...,False,False,False,True,-0.258819,0.965926,-0.5,0.866025,-2.449294e-16,1.00000


In [220]:
# ============================
# FIX STRUCTURAL ZERO ISSUES
# ============================

# Building size cannot be zero
df.loc[df['Building Size (m²)'] <= 0, 'Building Size (m²)'] = df['Building Size (m²)'].median()

# Occupancy cannot be zero for ratio calculations
df.loc[df['Occupancy Rate (%)'] <= 0, 'Occupancy Rate (%)'] = df['Occupancy Rate (%)'].median()

# Grid stability should not be zero
df.loc[df['Grid Stability Score'] <= 0, 'Grid Stability Score'] = df['Grid Stability Score'].median()

df.loc[df['Energy Consumption (kWh)'] <= 0, 'Energy Consumption (kWh)'] = df['Energy Consumption (kWh)'].median()


df.loc[df['Energy Consumption (kWh)'] < 5, 'Energy Consumption (kWh)'] = df['Energy Consumption (kWh)'].median()
print("Structural zero corrections complete.")


Structural zero corrections complete.


Now creating different features. I have divided features into 5 categories 
these are those categories 
Consumption Behavior

Structural Efficiency

Sustainability Indicators

Environmental & Comfort Stress

Smart Grid Interaction


Category 1. CONSUMPTION BEHAVIOR FEATURES
------------------------------------------

A) Energy Intensity (main)

In [221]:
df['Energy_Intensity'] = df['Energy Consumption (kWh)'] / (df['Building Size (m²)'] + 1e-6)

B) HVAC Dependency Ratio

In [222]:
df['HVAC_Ratio'] = df['HVAC Consumption (kWh)'] / (df['Energy Consumption (kWh)'] + 1e-6)

C) Lighting Dependency Ratio

In [223]:
df['Lighting_Ratio'] = df['Lighting Consumption (kWh)'] / (df['Energy Consumption (kWh)'] + 1e-6)

D) Plug Load Ratio

In [224]:
df['Plug_Ratio'] = df['Smart Plug Usage (kWh)'] / (df['Energy Consumption (kWh)'] + 1e-6)

E) Zonal HVAC Load Ratio

In [225]:
df['Zonal_Ratio'] = df['Zonal Heating/Cooling Data (kWh)'] / (df['Energy Consumption (kWh)'] + 1e-6)

Category 2. STRUCTURAL EFFICIENCY FEATURES
----------------------------------------

A) Occupancy Adjusted Energy

In [226]:
df['Occupancy_Adjusted_Energy'] = df['Energy Consumption (kWh)'] / (df['Occupancy Rate (%)'] + 1e-6)

B) Equipment Aging Index

In [227]:
df['Equipment_Aging_Index'] = df['Equipment Age (years)'] / (df['Building Age (years)'] + 1e-6)

C) Insulation Efficiency Proxy

In [228]:
df['Insulation_Efficiency'] = df['Insulation Quality Score'] / (df['Window-to-Wall Ratio (%)'] + 1e-6)

Category 3. SUSTAINABILITY FEATURES
-------------------------------------

A) Renewable Utilization Ratio

In [229]:
df['Renewable_Ratio'] = df['Local Energy Production (kWh)'] / (df['Energy Consumption (kWh)'] + 1e-6)

B) Carbon Load Index

In [230]:
df['Carbon_Load'] = df['Carbon Emission Rate (g CO2/kWh)'] * df['Energy Consumption (kWh)']

C) Efficiency Gap

In [231]:
df['Efficiency_Gap'] = 100 - df['Energy Efficiency Rating']
df.drop(columns=['Energy Efficiency Rating'], inplace=True)

D) Savings Gap

In [232]:
df['Savings_Gap'] = df['Energy Savings Target (%)'] - df['Energy Savings Potential (%)']

Category 4. ENVIRONMENTAL & COMFORT STRESS
----------------------------------------

A) Temperature Deviation

In [233]:
df['Temp_Deviation'] = abs(df['Temperature (°C)'] - df['Indoor Temperature (°C)'])

B) Climate Load Stress

In [234]:
df['Climate_Load'] = df['Solar Irradiance (W/m²)'] * df['Energy Consumption (kWh)']

Category 5. SMART ADAPTATION CAPABILITY
-------------------------------------

A) Smart Response Index

In [235]:
df['Smart_Response_Index'] = (
    df['IoT Sensor Count'] *
    df['Demand Response Participation']
)

B) Grid Stress Index

In [236]:
df['Grid_Stress'] = df['Energy Consumption (kWh)'] / (df['Grid Stability Score'] + 1e-6)

Some extra features as per need 

In [237]:
# Interaction features (simulate causal relationships)
df['HVAC_x_Temp'] = df['HVAC Consumption (kWh)'] * df['Temperature (°C)']
df['Lighting_x_Occupancy'] = df['Lighting Consumption (kWh)'] * df['Occupancy Rate (%)']

In [238]:
df['HVAC_x_Temp'] = np.sign(df['HVAC_x_Temp']) * np.log1p(np.abs(df['HVAC_x_Temp']))

In [239]:
df.drop(columns=[
    'Plug_Ratio',
    'Zonal_Ratio'
], inplace=True)

In [240]:
df['Energy_Intensity'] = np.log1p(df['Energy_Intensity'])
df['Lighting_x_Occupancy'] = np.log1p(df['Lighting_x_Occupancy'])

In [241]:
print(df[['HVAC_x_Temp','Lighting_x_Occupancy','Climate_Load']].describe())

        HVAC_x_Temp  Lighting_x_Occupancy  Climate_Load
count  52585.000000          52585.000000  52585.000000
mean       5.394677              6.162584  12016.888606
std        1.554168              1.144440   7209.465668
min       -6.279699              0.000000      0.000000
25%        5.161484              5.824528   6679.273403
50%        5.732754              6.410399  11037.400944
75%        6.162397              6.845357  16282.396936
max        7.480706              7.983913  54338.427001


Validate Engineered Features

In [242]:
print(df[['Energy_Intensity','HVAC_Ratio','Renewable_Ratio','Carbon_Load']].describe())

       Energy_Intensity    HVAC_Ratio  Renewable_Ratio    Carbon_Load
count      52585.000000  52585.000000     52585.000000   52585.000000
mean           0.066947      0.360349         0.124029   23412.523222
std            0.117448      0.305686         0.116008   12111.332217
min            0.002408      0.000000         0.000000     209.980571
25%            0.032387      0.195913         0.060370   14535.433472
50%            0.047080      0.296022         0.100797   21919.959570
75%            0.069555      0.430995         0.154488   30543.186421
max            4.739294      5.065625         2.317919  100960.617851


Correlation check

In [243]:
corr = df.corr().abs().unstack().sort_values(ascending=False)
corr = corr[corr < 1.0]   # exclude self-correlation
print(corr.head(20))

Grid_Stress                       Energy Consumption (kWh)            0.933623
Energy Consumption (kWh)          Grid_Stress                         0.933623
Demand Response Participation     Smart_Response_Index                0.921544
Smart_Response_Index              Demand Response Participation       0.921544
Solar Irradiance (W/m²)           Climate_Load                        0.752469
Climate_Load                      Solar Irradiance (W/m²)             0.752469
Energy Savings Target (%)         Savings_Gap                         0.710894
Savings_Gap                       Energy Savings Target (%)           0.710894
                                  Energy Savings Potential (%)        0.703310
Energy Savings Potential (%)      Savings_Gap                         0.703310
Carbon_Load                       Energy Consumption (kWh)            0.697194
Energy Consumption (kWh)          Carbon_Load                         0.697194
Carbon Emission Rate (g CO2/kWh)  Carbon_Load       

In [244]:


corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

high_corr = [column for column in upper.columns if any(upper[column] > 0.95)]

print("Highly correlated columns:", high_corr)

Highly correlated columns: []


In [245]:
print(df[['Energy_Intensity','HVAC_Ratio','Renewable_Ratio']].describe())

       Energy_Intensity    HVAC_Ratio  Renewable_Ratio
count      52585.000000  52585.000000     52585.000000
mean           0.066947      0.360349         0.124029
std            0.117448      0.305686         0.116008
min            0.002408      0.000000         0.000000
25%            0.032387      0.195913         0.060370
50%            0.047080      0.296022         0.100797
75%            0.069555      0.430995         0.154488
max            4.739294      5.065625         2.317919


In [246]:
df['Carbon_Load'] = np.log1p(df['Carbon_Load'])
df['Climate_Load'] = np.log1p(df['Climate_Load'])